# Goldenspike, an example of an end-to-end analysis using RAIL

author: Sam Schmidt, Eric Charles, Alex Malz, John Franklin Crenshaw, others...<br>
last run successfully: April 28, 2023 (with qp-prob >= 0.8.2)

This notebook demonstrates how to use a the various RAIL Modules to draw synthetic samples of fluxes by color, apply physical effects to them, train photo-Z estimators on the samples, test and validate the preformance of those estimators, and to use the RAIL summarization modules to obtain n(z) estimates based on the p(z) estimates.

### Creation 

Note that in the parlance of the Creation Module, "degradation" is any post-processing that occurs to the "true" sample generated by the create Engine.  This can include adding photometric errors, applying quality cuts, introducing systematic biases, etc.

In this notebook, we will draw both test and training samples from a RAIL Engine object. Then we will demonstrate how to use RAIL degraders to apply effects to those samples.

### Training and Estimation

The RAIL Informer modules "train" or "inform" models used to estimate p(z) given band fluxes (and potentially other information).

The RAIL Estimation modules then use those same models to actually apply the model and extract the p(z) estimates.

### p(z) Validation 

The RAIL Validator module applies various metrics 

### p(z) to n(z) Summarization

The RAIL Summarization modules convert per-galaxy p(z) posteriors to ensemble n(z) estimates. 

###  Imports

In [ ]:
# Prerquisites: os, numpy, pathlib, pzflow, tables_io
import os
import numpy as np
from pathlib import Path
from pzflow.examples import get_galaxy_data
import tables_io

In [ ]:
# Various rail modules
import rail
from rail.creation.degradation import LSSTErrorModel, InvRedshiftIncompleteness, LineConfusion, QuantityCut
from rail.creation.engines.flowEngine import FlowModeler, FlowCreator, FlowPosterior
from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter

from rail.estimation.algos.bpz_lite import Inform_BPZ_lite, BPZ_lite
from rail.estimation.algos.knnpz import Inform_KNearNeighPDF, KNearNeighPDF
from rail.estimation.algos.flexzboost import Inform_FZBoost, FZBoost

from rail.estimation.algos.naiveStack import NaiveStack
from rail.estimation.algos.pointEstimateHist import PointEstimateHist

from rail.evaluation.evaluator import Evaluator



RAIL now uses ceci as a back-end, which takes care of a lot of file I/O decisions to be consistent with other choices in DESC.

This bit effectively overrides a ceci default to prevent overwriting previous results, generally good but not necessary for this demo.

The `DataStore` uses `DataHandle` objects to keep track of the connections between the various stages.  When one stage returns a `DataHandle` and then you pass that `DataHandle` to another stage, the underlying code can establish the connections needed to build a reproducilble pipeline.   

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

Here we need a few configuration parameters to deal with differences in data schema between existing PZ codes.

In [ ]:
from rail.core.utils import RAILDIR
flow_file = os.path.join(RAILDIR, 'examples/goldenspike/data/pretrained_flow.pkl')
bands = ['u','g','r','i','z','y']
band_dict = {band:f'mag_{band}_lsst' for band in bands}
rename_dict = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

## Train the Flow Engine

First we need to train the normalizing flow that will serve as the engine for the notebook.

In the cell below, we load the example galaxy catalog from PZFlow and save it so that it can be used to train the flow. We also set the path where we will save the flow

In [ ]:
DATA_DIR = Path().resolve() / "data"
DATA_DIR.mkdir(exist_ok=True)

catalog_file = DATA_DIR / "base_catalog.pq"
catalog = get_galaxy_data().rename(band_dict, axis=1)
tables_io.write(catalog, str(catalog_file.with_suffix("")), catalog_file.suffix[1:])

catalog_file = str(catalog_file)
flow_file = str(DATA_DIR / "trained_flow.pkl")

Now we set the parameters for the FlowModeler, i.e. the pipeline stage that trains the flow:

In [ ]:
flow_modeler_params = {
    "name": "flow_modeler",
    "input": catalog_file,
    "model": flow_file,
    "seed": 0,
    "phys_cols": {"redshift": [0, 3]},
    "phot_cols": {
        "mag_u_lsst": [17, 35],
        "mag_g_lsst": [16, 32],
        "mag_r_lsst": [15, 30],
        "mag_i_lsst": [15, 30],
        "mag_z_lsst": [14, 29],
        "mag_y_lsst": [14, 28],
    },
    "calc_colors": {"ref_column_name": "mag_i_lsst"},
}

Now we will create the flow and train it

In [ ]:
flow_modeler = FlowModeler.make_stage(**flow_modeler_params)

In [ ]:
flow_modeler.fit_model()

## Make mock data

Now we will use the trained flow to create training and test data for the photo-z estimators.

For both the training and test data we will:

1. Use the Flow to produce some synthetic data
2. Use the LSSTErrorModel to add photometric errors
3. Use the FlowPosterior to estimate the redshift posteriors for the degraded sample
4. Use the ColumnMapper to rename the error columns so that they match the names in DC2.
5. Use the TableConverter to convert the data to a numpy dictionary, which will be stored in a hdf5 file with the same schema as the DC2 data

### Training sample

For the training data we are going to apply a couple of extra degradation effects to the data beyond what we do to create test data, as the training data will have some spectroscopic incompleteness.  This will allow us to see how the trained models perform with imperfect training data.

More details about the degraders are available in the `rail/examples/creation_examples/degradation_demo.ipynb` notebook.


In [ ]:
flow_creator_train = FlowCreator.make_stage(
    name='flow_creator_train', 
    model=flow_modeler.get_handle("model"), 
    n_samples=50,
    seed=1235,
)

lsst_error_model_train = LSSTErrorModel.make_stage(
    name='lsst_error_model_train',
    bandNames=band_dict, 
    seed=29,
)

inv_redshift = InvRedshiftIncompleteness.make_stage(
    name='inv_redshift',
    pivot_redshift=1.0,
)

line_confusion = LineConfusion.make_stage(
    name='line_confusion', 
    true_wavelen=5007., 
    wrong_wavelen=3727.,
    frac_wrong=0.05,
)

quantity_cut = QuantityCut.make_stage(
    name='quantity_cut',    
    cuts={'mag_i_lsst': 25.0},
)

col_remapper_train = ColumnMapper.make_stage(
    name='col_remapper_train', 
    columns=rename_dict,
)
   
table_conv_train = TableConverter.make_stage(
    name='table_conv_train', 
    output_format='numpyDict',
)

In [ ]:
train_data_orig = flow_creator_train.sample(150, 1235)
train_data_errs = lsst_error_model_train(train_data_orig,seed=66)
train_data_inc = inv_redshift(train_data_errs)
train_data_conf = line_confusion(train_data_inc)
train_data_cut = quantity_cut(train_data_conf)
train_data_pq = col_remapper_train(train_data_cut)
train_data = table_conv_train(train_data_pq)

Let's examine the quantities that we've generated, we'll use the handy `tables_io` package to temporarily write to a pandas dataframe for quick writeout of the columns:

In [ ]:
train_table = tables_io.convertObj(train_data.data, tables_io.types.PD_DATAFRAME)
train_table.head()

You see that we've generated redshifts, ugrizy magnitudes, and magnitude errors with names that match those in the cosmoDC2_v1.1.4_image data.

### Testing sample

For the test sample we will:

1. Use the Flow to produce some synthetic data
2. Use the LSSTErrorModel to smear the data
3. Use the FlowPosterior to estimate the redshift posteriors for the degraded sample
4. Use ColumnMapper to rename some of the columns to match DC2
5. Use the TableConverter to convert the data to a numpy dictionary, which will be stored in a hdf5 file with the same schema as the DC2 data

In [ ]:
flow_creator_test = FlowCreator.make_stage(
    name='flow_creator_test',
    model=flow_modeler.get_handle("model"),
    n_samples=50,
)
      
lsst_error_model_test = LSSTErrorModel.make_stage(
    name='lsst_error_model_test',
    bandNames=band_dict,
)

flow_post_test = FlowPosterior.make_stage(
    name='flow_post_test',
    model=flow_modeler.get_handle("model"),
    column='redshift',
    grid=np.linspace(0., 5., 21),
)
                
col_remapper_test = ColumnMapper.make_stage(
    name='col_remapper_test',
    columns=rename_dict,
    hdf5_groupname='',
)

table_conv_test = TableConverter.make_stage(
    name='table_conv_test', 
    output_format='numpyDict',
)


In [ ]:
test_data_orig = flow_creator_test.sample(150, 1234)
test_data_errs = lsst_error_model_test(test_data_orig,seed=58)
test_data_post = flow_post_test.get_posterior(test_data_errs, err_samples=None)
test_data_pq = col_remapper_test(test_data_errs)
test_data = table_conv_test(test_data_pq)

In [ ]:
test_table = tables_io.convertObj(test_data.data, tables_io.types.PD_DATAFRAME)
test_table.head()

## "Inform" some estimators

More details about the process of "informing" or "training" the models used by the estimators is available in the `rail/examples/estimation_examples/RAIL_estimation_demo.ipynb` notebook.

We use "inform" rather than "train" to generically refer to the preprocessing of any prior information.
For a machine learning estimator, that prior information is a training set, but it can also be an SED template library for a template-fitting or hybrid estimator.

In [ ]:
inform_bpz = Inform_BPZ_lite.make_stage(
    name="inform_bpz",
    model="bpz.pkl",
    hdf5_groupname="",
)

inform_knn = Inform_KNearNeighPDF.make_stage(
    name='inform_knn', 
    nondetect_val=np.nan,
    model='knnpz.pkl', 
    hdf5_groupname='',
)

inform_fzboost = Inform_FZBoost.make_stage(
    name='inform_FZBoost', 
    model='fzboost.pkl', 
    hdf5_groupname='',
)

In [ ]:
inform_bpz.inform(train_data)
inform_knn.inform(train_data)
inform_fzboost.inform(train_data)


**If you run into issues here:** 

If you've installed rail and bpz to different directories (most commonly, you've installed rail from 
source and bpz from PyPI), you may run into an issue where rail cannot locate a file installed by bpz.

To fix this, find your test_bpz.columns file in your bpz directory ([or grab a new one here on 
GitHub](https://github.com/LSSTDESC/rail_bpz/blob/main/src/rail/examples/estimation/configs/test_bpz.columns>)) 
and copy it into your rail directory to `/RAIL/src/rail/examples/estimation_examples/configs/test_bpz.columns`. 

Alternatively, if you don't want to move files, you should be able rewrite code to replace the configured paths with 
your actual `test_bpz.columns` path (inform stage: [bpz_lite.py L89](https://github.com/LSSTDESC/rail_bpz/blob/65870ffd93ba35356a1af44104a0a78530085789/src/rail/estimation/algos/bpz_lite.py#L89) and estimation: [bpz_lite.py L259](https://github.com/LSSTDESC/rail_bpz/blob/65870ffd93ba35356a1af44104a0a78530085789/src/rail/estimation/algos/bpz_lite.py#L259>)).



## Estimate photo-z posteriors

More details about the estimators is available in the `rail/examples/estimation_examples/RAIL_estimation_demo.ipynb` notebook.

`randomPZ` is a very simple class that does not actually predict a meaningful photo-z, instead it produces a randomly drawn Gaussian for each galaxy.<br>
`trainZ` is our "pathological" estimator, it makes a PDF from a histogram of the training data and assigns that PDF to every galaxy.<br>
`BPZ_lite` is a template-based code that outputs the posterior estimated given a specific template set and Bayesian prior. See Benitez (2000) for more details.<br>


In [ ]:
estimate_bpz = BPZ_lite.make_stage(
    name='estimate_bpz', 
    hdf5_groupname='', 
    model=inform_bpz.get_handle('model'),
)

estimate_knn = KNearNeighPDF.make_stage(
    name='estimate_knn', 
    hdf5_groupname='', 
    nondetect_val=np.nan, 
    model=inform_knn.get_handle('model'),
)

estimate_fzboost = FZBoost.make_stage(
    name='test_FZBoost', 
    nondetect_val=np.nan,
    model=inform_fzboost.get_handle('model'), 
    hdf5_groupname='',
    aliases=dict(input='test_data', output='fzboost_estim'),
)

In [ ]:
knn_estimated = estimate_knn.estimate(test_data)
fzboost_estimated = estimate_fzboost.estimate(test_data)
bpz_estimated = estimate_bpz.estimate(test_data)

## Evaluate the estimates

Now we evaluate metrics on the estimates, separately for each estimator.  

Each call to the `Evaluator.evaluate` will create a table with the various performance metrics. 
We will store all of these tables in a dictionary, keyed by the name of the estimator.

In [ ]:
eval_dict = dict(bpz=bpz_estimated, fzboost=fzboost_estimated, knn=knn_estimated)
truth = test_data_orig

result_dict = {}
for key, val in eval_dict.items():
    the_eval = Evaluator.make_stage(name=f'{key}_eval', truth=truth)
    result_dict[key] = the_eval.evaluate(val, truth)

The Pandas DataFrame output format conveniently makes human-readable printouts of the metrics.  
This next cell will convert everything to Pandas.

In [ ]:
results_tables = {key:tables_io.convertObj(val.data, tables_io.types.PD_DATAFRAME) for key,val in result_dict.items()}

In [ ]:
results_tables['knn']

In [ ]:
results_tables['fzboost']

In [ ]:
results_tables['bpz']

## Summarize the per-galaxy redshift constraints to make population-level distributions

{introduce the summarizers}

First we make the stages, then execute them, then plot the output.

In [ ]:
point_estimate_test = PointEstimateHist.make_stage(name='point_estimate_test')
naive_stack_test = NaiveStack.make_stage(name='naive_stack_test')

In [ ]:
point_estimate_ens = point_estimate_test.summarize(eval_dict['bpz'])
naive_stack_ens = naive_stack_test.summarize(eval_dict['bpz'])

In [ ]:
_ = naive_stack_ens.data.plot_native(xlim=(0,3))

In [ ]:
_ = point_estimate_ens.data.plot_native(xlim=(0,3))

### Convert this to a `ceci` Pipeline

Now that we have all these stages defined and configured, and that we have established the connections between them by passing `DataHandle` objects between them, we can build a `ceci` Pipeline.


In [ ]:
import ceci
pipe = ceci.Pipeline.interactive()
stages = [
    # train the flow
    flow_modeler,
    # create the training catalog
    flow_creator_train, lsst_error_model_train, inv_redshift,
    line_confusion, quantity_cut, col_remapper_train, table_conv_train,
    # create the test catalog
    flow_creator_test, lsst_error_model_test, col_remapper_test, table_conv_test,
    # inform the estimators
    inform_bpz, inform_knn, inform_fzboost,
    # estimate posteriors
    estimate_bpz, estimate_knn, estimate_fzboost,
    # estimate n(z), aka "summarize"
    point_estimate_test, naive_stack_test,
]
for stage in stages:
    pipe.add_stage(stage)

In [ ]:
pipe.initialize(dict(input=catalog_file), dict(output_dir='.', log_dir='.', resume=False), None)

In [ ]:
pipe.save('tmp_goldenspike.yml')

### Read back the pipeline and run it

In [ ]:
pr = ceci.Pipeline.read('tmp_goldenspike.yml')

In [ ]:
pr.run()

# Clean up:

Finally, you'll notice that we've written a large number of temporary files in the course of running this demo, to delete these and clean up the directory just run the `cleanup.sh` script in this directory to delete the data files.